<div style="float:left">
    <h1 style="width:600px">Technical assessment</h1>
    <h3 style="width:600px">Solution Engineer position</h3>
    <h3 style="width:600px">Author: Andres Restrepo</h3>

</div>
<div style="float: right; display: flex; align-items: center;">
    <img width="260" src="Logo/logo_CARTO_positive_180.png" />
</div>

## Libraries

In [ ]:
#pip install rarfile

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import matplotlib.pyplot as plt

import zipfile
import rarfile
#import seaborn as sns

In [ ]:
# # Setting theme
# sns.set_theme()

# Data

## Download

### Jaguar records

The jaguar records data is downloaded from the CARTO stack platform.

In [ ]:
%%time
with zipfile.ZipFile('Data/Jag/carto-data.zip', 'r') as zip_file:
    with zip_file.open('puma_concolor.csv') as file:
        # read data as datafame
        raw_jag = pd.read_csv(file,low_memory=False)

In [ ]:
# Raw data reading
raw_jag.head()

In [ ]:
# Data review
raw_jag.info()

### Municipalties data

The municiplities boundaries data from Colombia is downloaded from the National Geostatistical Framework [(*Marco Geoestadistico Nacional*)](https://geoportal.dane.gov.co/servicios/descarga-y-metadatos/descarga-mgn-marco-geoestadistico-nacional/#gsc.tab=0) , published by the Colombian National Administrative and Statisticas Department [(*Departamento Administrativo Nacional de Estadistica*)](https://www.dane.gov.co/).

Colombian municipalites data: [Shapefile](https://geoportal.dane.gov.co/descargas/mgn_2021/MGN2021_MPIO_POLITICO.rar)

In [ ]:
geo_mun = gpd.read_file('Data/Mun/MGN_MPIO_POLITICO.shp')

In [ ]:
# Raw data reading
geo_mun.head()

In [ ]:
# Data review
geo_mun.info()

In [ ]:
# Geometry
geo_mun.plot() 

In [ ]:
# CRS
geo_mun.crs

### Natural parks data

The national parks data represents the boundaries of restricted and conservations areas under the management of central government envirnmental authorities. Municipalities with area within these limits would have restricted area to host the jaguar ecosystem and greater support from the central government in that matter.

The national park boundaries data is available in [ESRI Open Data Hub](https://datosabiertos.esri.co/datasets/d4d80793ff604f7aa153f3cecbe0757e/explore).

In [ ]:
geo_park = gpd.read_file('Data/Parks/Parques_Nacionales_Naturales_de_Colombia.shp')

In [ ]:
# Raw data reading
geo_park.head()

In [ ]:
# Data review
geo_park.info()

In [ ]:
# Geometry
geo_park.plot() 

In [ ]:
# CRS
geo_park.crs

## Reading

### Jaguar records

In [ ]:
# Set geometry
#geometry = gpd.points_from_xy(raw_jag['longitude'], raw_jag['latitude'])

In [ ]:
# Reading geopandas df
geo_jag = gpd.GeoDataFrame(raw_jag, geometry=gpd.points_from_xy(raw_jag.longitude, raw_jag.latitude), crs="EPSG:4326")

In [ ]:
# Data reading
geo_jag.head()

In [ ]:
# Data review
geo_jag.info()

In [ ]:
# Geometry
geo_jag.plot() 

In [ ]:
# Setting CRS
geo_jag = geo_jag.to_crs(crs="EPSG:4686")

In [ ]:
# Review new CRS
geo_jag.crs

### Natural parks data

In [ ]:
# Setting CRS
geo_park = geo_park.to_crs(crs="EPSG:4686")

In [ ]:
# Review new CRS
geo_park.crs

## Subsetting

### Spatial subsetting

In [ ]:
geo_jag_col = gpd.clip(geo_jag, geo_mun)

In [ ]:
geo_jag_col.shape

In [ ]:
geo_jag_col.head()

In [ ]:
# Geometry
fig, ax = plt.subplots(1,1, figsize=(12,9))
geo_jag_col.plot(ax=ax) 

## Spatial aggregation

### Record count

In [ ]:
# Create a record count column
geo_mun['record_count'] = 0

In [ ]:
# Count number of record in each municipality
for index, mun in geo_mun.iterrows():
    # Filter points that intersect with the current polygon
    intersecting_points = geo_jag_col[geo_jag_col.intersects(mun.geometry)]
    
    # Count the number of intersecting points
    count = len(intersecting_points)
    
    # Store the count in the 'point_count' column
    geo_mun.loc[index, 'record_count'] = count



In [ ]:
geo_mun.head()

In [ ]:
geo_mun['record_count'].describe()

In [ ]:
# Plotting by record count
fig, ax = plt.subplots(1,1, figsize=(8,5),tight_layout=True)
plt.suptitle('Jaguar record data distribution by municipality in Colombia',fontsize=18)
geo_mun.hist(column='record_count',ax=ax,bins=75)

# Titles
ax.set_title("Record count",fontsize=12)

In [ ]:
# Municipality with highest value of cases
geo_mun[geo_mun.record_count == geo_mun.record_count.max()]

In [ ]:
# Plotting by record count
fig, ax = plt.subplots(1,1, figsize=(12,9))
geo_mun.plot(column='record_count',ax=ax)

### Record rate

The jaguar record rate (records/area) is a result of the relation of the number of records and the area of each municipality (squared meteres).

__Record date =__ (number of cases) / (squared meters) 

In [ ]:
# Creates the new feature
geo_mun['record_rate'] = 0

In [ ]:
geo_mun['record_rate'] = (geo_mun.record_count)/(geo_mun.MPIO_NAREA)

In [ ]:
geo_mun['record_rate'].describe()

In [ ]:
# Municipality with highest rate
geo_mun[geo_mun.record_rate == geo_mun.record_rate.max()]

In [ ]:
# Plotting by record count
fig, ax = plt.subplots(1,1, figsize=(12,9))
geo_mun.plot(column='record_rate',ax=ax)

## National parks

### Area

In [ ]:
# Intersection between mun and park
# intersection_parks = gpd.overlay(geo_mun, geo_park, how='intersection')

**Review resulting geometry**

In [ ]:
# # Plotting
# fig, ax = plt.subplots(1,1, figsize=(12,9))
# intersection_parks.plot(ax=ax)

In [ ]:
# intersection_parks.head(1)

In [ ]:
# # Plotting
# fig, ax = plt.subplots(1,1, figsize=(12,9))
# intersection_parks.head(1).plot(ax=ax)

In [ ]:
# geo_mun[geo_mun.MPIO_CDPMP == '05004']

In [ ]:
# fig, ax = plt.subplots(1,1, figsize=(12,9))
# geo_mun[geo_mun.MPIO_CDPMP == '05004'].plot(ax=ax)

In [ ]:
# Function for calculatins intercepting area percentage
def calculate_intersection_percentage(initial_df, intersecting_df, new_column_name):
    # Create a new column in the initial DataFrame
    initial_df[new_column_name] = None

    # Iterate over each polygon in the initial DataFrame
    for idx, polygon in initial_df.iterrows():
        intersection = initial_df.geometry[idx].intersection(intersecting_df.unary_union)
        intersection_area = intersection.area
        percentage = (intersection_area / initial_df.geometry[idx].area) * 100
        initial_df.at[idx, new_column_name] = float(percentage)

    return initial_df

In [ ]:
geo_mun = calculate_intersection_percentage(geo_mun, geo_park, 'percentage_area_park')

### Distance

In [ ]:
geo_mun.info()